In [125]:
from collections import defaultdict
from itertools import combinations, product
import numpy as np
import pandas as pd

---

#### Integers and their binary representations

In [126]:
def dec_to_bin_repr (a, n):
    """return the binary representation of an integer as a string to n positions"""
    return format(a, '0'+str(n)+'b')
print(dec_to_bin_repr(17, 10))

0000010001


In [127]:
def bin_repr_inv (a, n):
    """return the inverse of the binary representation of an integer as a string to n positions"""
    return format(a^(2**n-1), '0'+str(n)+'b')
print(bin_repr_inv(17, 10))

1111101110


---

In [128]:
def vec_weight (vector):
    """take a vector of length n as a string and return its weight as an integer where the weight of a vector is the number of nonzero entries in the vector"""
    return np.sum([1 for i in vector if i != '0'])
vectorized_vec_weight = np.vectorize(vec_weight)

vec_weight('0000010001')

2

In [129]:
def vector_sum (vectors, q):
    """take a set of vectors as a list of strings of length n and return their sum (mod q) as a string"""
    
    # verify that all vectors are of length n
    for vector in vectors:
        assert len(vector) == len(vectors[0]), 'vectors must be of equal length'
        
    positions = defaultdict(list)
    for vector in vectors:
        for i in range(len(vector)):
            positions[i].append(vector[i])
    return ''.join([str(i) for i in [np.mod(np.array(j).astype(int).sum(), q) for i, j in positions.items()]])

#def vector_add (a, b, q):
#    """take two vectors a and b of length n as strings and return their sum (mod q) as a string"""
     # verify that both vectors are of length n
#    assert len(a) == len(b), 'vectors must be of equal length' 
#    return ''.join([str((int(a[i]) + int(b[i])) % q) for i in range(len(a))])
     #return format(a^b, '0'+str(n)+'b') for binary vectors only

vector_sum(['0121','2210','0101'], 3)

'2102'

In [106]:
def v (n, q):
    """construct a linear space of dimension n over a finite field of order q"""
    return [''.join(str(pos) for pos in comb) for comb in list(product(range(q), repeat=n))]

def num_lin_sub_k (n, k, q):
    """Compute the number of k-dim linear subspaces of a linear space of dim n over a finite field of order q"""
    return np.prod([q**n-q**i for i in range(k)])/np.prod([q**k-q**i for i in range(k)])

def generate_scalar_multiples (vector, q):
    """Given a vector of length n over q symbols, get its scalar multiples"""
    scalar_multiples = []
    for scalar in range(q):
        scalar_multiple = vector
        for i in range(scalar):
            scalar_multiple = vector_sum([scalar_multiple, vector], q)
        scalar_multiples.append(scalar_multiple)
    return scalar_multiples

def span (vectors, q):
    """get the span of a set of vectors in V(n, q) over GF(q)"""
    
    # verify that all vectors are of length n
    for vector in vectors:
        assert len(vector) == len(vectors[0]), 'vectors must be of equal length'
    
    # remove the zero vector from the set of vectors if it is contained in them
    if '0'*len(vectors[0]) in vectors:
        vectors.remove('0'*len(vectors[0]))
        
    qsm = []
    for veck in vectors:
        qsm.append(generate_scalar_multiples(veck, q))

    return set([vector_sum(comb, q) for comb in product(*[qsm[i] for i in range(len(qsm))])])

def gen_dis_bases (space, k):
    if k == 0:
        return [{0}], 0
    
    bases = list(combinations(space[1:], k))
    bases = list(set(i) for i in bases)
    #print(len(bases))
    #for i in bases:
    #    print(i)
            
    spans = defaultdict(list)
    while bases:
        basis, bases = bases[0], bases[1:]
        
        # check if the set of vectors is linearly independent
        if len(span(basis, q)) != q**k:
            continue
        
        s = span(basis, q)
        tot = ''
        for i in sorted(s):
            tot += i
        spans[tot].append(basis)
        
    return len(spans), [len(reprs) for spn, reprs in spans.items()], spans

def get_basis (space, k):
    num, l, bases = gen_dis_bases(space, k)
    return np.random.choice(bases[np.random.choice(np.array([spn for spn, reprs in bases.items()]))])

def linear_code (basis, q):
    """given a set of basis vectors, return a k-dimensional linear code of length n"""
    return span(basis, q)

---

### Construct $V(n, q)$ over $GF(q)$ and compute the scalar multiples of a vector $\{\lambda\mathbf{v}\,\vert\,\mathbf{v}\in V(n, q), \forall \lambda\in GF(q)\}$

$| V(n, q) | = q^n$

In [62]:
n = 5           # specify the dimension of V(n, q)
q = 2           # specify the order of GF(q)
space = v(n, q) # construct V(n, q) over GF(q)

print('V(' + str(n) + ', ' + str(q) + ')')
print()
print('{}\t{}\t{}'.format(str(n**q) + ' vectors', 'weight', str(q) + ' scalar multiples per vector'))
print()
for i, vector in enumerate(space):
    print('{}\t{}\t{}\t{}'.format(i, vector, vector_weight(vector), generate_scalar_multiples(vector, q)))

V(5, 2)

25 vectors	weight	2 scalar multiples per vector

0	00000	0.0	['00000', '00000']
1	00001	1	['00001', '00000']
2	00010	1	['00010', '00000']
3	00011	2	['00011', '00000']
4	00100	1	['00100', '00000']
5	00101	2	['00101', '00000']
6	00110	2	['00110', '00000']
7	00111	3	['00111', '00000']
8	01000	1	['01000', '00000']
9	01001	2	['01001', '00000']
10	01010	2	['01010', '00000']
11	01011	3	['01011', '00000']
12	01100	2	['01100', '00000']
13	01101	3	['01101', '00000']
14	01110	3	['01110', '00000']
15	01111	4	['01111', '00000']
16	10000	1	['10000', '00000']
17	10001	2	['10001', '00000']
18	10010	2	['10010', '00000']
19	10011	3	['10011', '00000']
20	10100	2	['10100', '00000']
21	10101	3	['10101', '00000']
22	10110	3	['10110', '00000']
23	10111	4	['10111', '00000']
24	11000	2	['11000', '00000']
25	11001	3	['11001', '00000']
26	11010	3	['11010', '00000']
27	11011	4	['11011', '00000']
28	11100	3	['11100', '00000']
29	11101	4	['11101', '00000']
30	11110	4	['11110', '00000']
31	11111	5	['11111',

In [4]:
# show that the sum of any two vectors in the linear space is also in the linear space

a = np.random.choice(space) # choose a random vector in V(n, q)
b = np.random.choice(space) # choose a random vector in V(n, q)
c = vecadd(a, b, n, q)      # add them
print(a)
print(b)
print(c)
assert c in space, 'Error: the sum of the vectors both of which are in V(n, q) is not in V(n, q)'

10011
11100
01111


---

### Compute the sum $\mathbf{v_1} + ... + \mathbf{v_k}$ of a set of vectors $\{\mathbf{v_1}, ..., \mathbf{v_k}\} \in V(n, q)$

In [6]:
num_vecs = np.random.randint(1, 5)
sum_vecs = np.random.choice(space, size=num_vecs, replace=True)
summation = vecsadd(sum_vecs, n, q)

print('Summing {} vector{}\t{}'.format(num_vecs, 's' if num_vecs != 1 else '', ' + '.join(sum_vecs)))
print('Sum\t\t\t{}'.format(summation))

Summing 3 vectors	11100 + 00101 + 11010
Sum			00011


### Get the span $span\{\mathbf{v_1}, ..., \mathbf{v_k}\}$ of a set of vectors $\{\mathbf{v_1}, ..., \mathbf{v_k}\} \in V(n, q)$

In [7]:
span_vecs = list(np.random.choice(space[1:], size=num_vecs, replace=False))
spn = span(span_vecs, n, q)

print('Spanning {} vector{}\t{}'.format(num_vecs, 's' if num_vecs != 1 else '', ' + '.join(span_vecs)))
print('Span {} vector{}\t\t{}'.format(len(spn), 's' if num_vecs != 1 else '', spn))

Spanning 3 vectors	01001 + 11101 + 00001
Span 8 vectors		{'10101', '01001', '11100', '10100', '00001', '01000', '00000', '11101'}


---

### Compute the number of distinct k-dim linear subspaces

The number of distinct linear subspaces of dim $k$ of a linear space of dim $n$ is $\frac{(q^n - q^0)(q^n - q^1)...(q^n - q^{k - 1})}{(q^k - q^0)(q^k - q^1)...(q^k - q^{k - 1})}$

In [10]:
# Compute the total number of linear subspaces of a linear space of dim n
# i.e., compute the number of k-dim linear subspaces for k = 1, ..., n
# the "0-dim" linear subspace is the trivial subspace {0}

print('{}\t{}\t{}\t{}'.format('k', 'q^k', 'bases', 'q^n_choose_k'))
print()
for k in range(0, n + 1):
    print('{}\t{}\t{}\t{}'.format(k, q**k, int(num_lin_sub_k(n, k, q)), len(list(combinations(range(q**n), k)))))

k	q^k	bases	q^n_choose_k

0	1	1	1
1	2	31	32
2	4	155	496
3	8	155	4960
4	16	31	35960
5	32	1	201376


---

### Compute distinct k-dim bases and their representations

In [236]:
bases = gen_dis_bases(space, k=3)

TypeError: 'set' object is not subscriptable

---

### Get a k-dim basis

In [105]:
basis = get_basis(space, k=3)
basis

TypeError: span() takes 2 positional arguments but 3 were given

---

### Generate a linear [n, k]-code

In [55]:
for i in sorted(gen_lin_code(basis, n, q)):
    print(i)

00000
00001
01110
01111
10000
10001
11110
11111


### Generate a linear [n, k]-code with a generator matrix

In [238]:
n=3
q=3
messages = np.array(list(list(i) for i in v(n, q)), dtype=int)
messages

array([[0, 0, 0],
       [0, 0, 1],
       [0, 0, 2],
       [0, 1, 0],
       [0, 1, 1],
       [0, 1, 2],
       [0, 2, 0],
       [0, 2, 1],
       [0, 2, 2],
       [1, 0, 0],
       [1, 0, 1],
       [1, 0, 2],
       [1, 1, 0],
       [1, 1, 1],
       [1, 1, 2],
       [1, 2, 0],
       [1, 2, 1],
       [1, 2, 2],
       [2, 0, 0],
       [2, 0, 1],
       [2, 0, 2],
       [2, 1, 0],
       [2, 1, 1],
       [2, 1, 2],
       [2, 2, 0],
       [2, 2, 1],
       [2, 2, 2]])

In [256]:
basis = ['0010', '0100', '1000']
generator = np.array(list(list(i) for i in basis), dtype=int)
generator

array([[0, 0, 1, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0]])

In [245]:
np.mod(np.dot(messages, generator), q)

array([[0, 0, 0, 0],
       [1, 0, 0, 0],
       [2, 0, 0, 0],
       [0, 1, 0, 0],
       [1, 1, 0, 0],
       [2, 1, 0, 0],
       [0, 2, 0, 0],
       [1, 2, 0, 0],
       [2, 2, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 1, 0],
       [2, 0, 1, 0],
       [0, 1, 1, 0],
       [1, 1, 1, 0],
       [2, 1, 1, 0],
       [0, 2, 1, 0],
       [1, 2, 1, 0],
       [2, 2, 1, 0],
       [0, 0, 2, 0],
       [1, 0, 2, 0],
       [2, 0, 2, 0],
       [0, 1, 2, 0],
       [1, 1, 2, 0],
       [2, 1, 2, 0],
       [0, 2, 2, 0],
       [1, 2, 2, 0],
       [2, 2, 2, 0]])

---

get_cosets (there are $q^{n-k}$ cosets)
1. each codeword of the linear code is the leader of its own group
2. for i in $q^{n-k}$
  1. 

In [255]:
def gen_cosets (C, q):
    """take a linear subspace of V(n, q) over GF(q) and return the cosets of the linear space"""
    space = v(len(list(C)[0]), q)
    space = sorted(space, key=lambda x: vector_weight(x))
    
    space = sorted(space, key=lambda x: int(x), reverse=True)
    cosets = defaultdict(list)
    
    # the first coset is the linear subspace itself
    C = sorted(C, key=lambda x: vector_weight(x))
    cosets[C[0]].extend(C[1:])
    for codeword in C:
        space.remove(codeword)
        
    while space:
        temp = []
        vector = space.pop(0)
        for codeword in C:
            vs = vector_sum([codeword, vector], q)
            temp.append(vs)
            if vs in space:
                space.remove(vs)
        #temp = sorted(temp, key=lambda x: vector_weight(x))
        
        cosets[temp[0]].extend(temp[1:])
        if vector in space:
            space.remove(vector)
    return cosets

q=3
cosets=gen_cosets(linear_code(['1011','0101'], q), q)
cosets
df = pd.DataFrame((k, *v) for k, v in cosets.items())
df

,0,1,2,3,4,5,6,7,8
0,0000,0101,0202,2120,1011,2022,1210,1112,2221
1,2222,2020,2121,1012,0200,1211,0102,0001,1110
2,2220,2021,2122,1010,0201,1212,0100,0002,1111
3,2212,2010,2111,1002,0220,1201,0122,0021,1100
4,2211,2012,2110,1001,0222,1200,0121,0020,1102
5,2210,2011,2112,1000,0221,1202,0120,0022,1101
6,2202,2000,2101,1022,0210,1221,0112,0011,1120
7,2201,2002,2100,1021,0212,1220,0111,0010,1122
8,2200,2001,2102,1020,0211,1222,0110,0012,1121
